## 4) Machine learning 

><font color='lightgrey'>GOAL OF THIS SECTION  
    *To compare different approaches for solving the machine learning problem.  
    Using different features spaces, different model types, different methods for tuning the models*  </font>

### (a) Our project goal phrased as a clear machine learning question

><font color='lightgrey'>GOAL OF THIS SECTION  
    *To state the intended outcome in machine learning terms.  
    To give the features and the target variable we are using.  
    To categorize our ML question as a regression or a classification task*  </font>

We want to be able to predict the cost of a new production order (our target) with a certain accuracy level by using the limited number information available for a new order at project start (our features).

The target is the 'cout_unitaire_pour_la_qte_demandee'

The features have been described in the preliminary chapter and gathered in the list used_features.
Those features correspond to parameters which are typically immediately available within a basic technical folder received at project start.  

This is a supervised regression task, as we try to predict a continuous value attribute by training our model with labeled data.

### (b) What models are involved and why?

><font color='lightgrey'>GOAL OF THIS SECTION  
    *To state which models we plan to use and why (model interpretability, suitability, scalability, diversity, …).  
    We use a ranking of our approaches: priority, optional or "nice to add"*  </font>

For this task we will compare an average baseline model to the models described inthe sections below.

Each time we try to observe our resulting metrics and check if the model is neither underfitting nor overfitting.  
To avoid over- or underfitting we will manage the bias/variance trade-off, with tuning model hyperparameters with grid search.  

#### Linear models
A multi-linear regression model with regularisation: ridge model. This will be a simple model to capture linear relationships between features and target.  

#### Non-Linear models
Non-linear models to capture non-linear patterns in features:  

- Nearest neighbors: we believe it could act well by grouping similar parts, we could also visualize the nearest neighbors (for instance display the corresponding pdf) which is a usefull information for the tender manager, and which would also us to better interpret the results.

- Random forest regression: we want to use this robust methodology which performs segmentation of the dataset with if/else rules: the boosting and bagging ensemble methods integrated in the model reduce respectively bias and variance of our decision trees estimators. The benefit of this method is to have a simple model with no hyperparameter having impact on the model complexity, meaning that we do not need to tune the model and we avoid any potential error in the implementation. It will be a good baseline for our non linear models. Random forrest are usually also used for larged datasets as beeing computionaly efficient; in out case with this small dataset, this effect is certainly usefull but not really decisive for the model choice. Also a very informative output from the model will be the possibility to perform classification of the features importance with the .feature_importance_ method. Plotting those will help us to confirm our preliminary assumptions on the importance of the features.

- Support vector regression with rbf kernel: should still be effective when dimensionality is greater than cardinality, which might be our case if we use all 3 sources of information we have). With this model 

### (c) Detailed machine learning strategy 

><font color='lightgrey'>*Preprocessing steps of your data for each machine learning model  
Methodologies to be used to train and finetune your models  
Your baseline model  
The metrics and methodologies you are considering to evaluate and compare your models  
You should clearly justify any of the above  
Feel free to use models that have not been covered in the course. We might ask you to implement a simple version of the model to make sure that you are suitably familiar with this model ahead of starting with the project.*  </font>

We start by setting up the notebook with necessary libraries

In [ ]:
# Import necessary library

import pandas as pd
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 150)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 150)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import scipy.stats as stats
import warnings
warnings.simplefilter('ignore',\
         category=(FutureWarning,DeprecationWarning,RuntimeWarning))
import pandas_profiling
from pandas_profiling import ProfileReport

from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error as MAE,mean_squared_error as MSE

from sklearn import set_config
set_config(display='diagram')

In [ ]:
# Creating variable for path name to the extracted data
#devis_path='C:/Users/BS/cpstdata/devis'

# Store this variable for usage in other notebooks
#%store devis_path

# Getting stored data back from original notebooks where data is generated

# Clean dataset input to ML after EDA
%store -r df_clean_scrome
%store -r df_clean_other

# original dataset to retrieve information about (for instance about pdf path)
%store -r df_merged
%store -r quant_features
%store -r bool_features
%store -r cat_features
%store -r cpst_path
%store -r git_path

#Checking the variables acctive in the notebook
%whos

In [ ]:
# Setting-up triggers to control the quantity of notebook outputs

profiling_trigger = 1   # Allow profiling reports of the datasets
plotting_trigger = 1   # Allow plots of the datasets

#### Features selection

We will not integrate feature selection in our ML models and we explain why in the section below:
- The EDA does not show outstanding features, rather a sum of many small equivalent contributions. We think that removing some of those could be immediatly detrimental to the ML model predicting capability, in the same proportion as quantity of features removed.  
- We currently do ot have a very high dimensionality and we are not at this stage in a logic of improving estimators’ accuracy scores or boost their performance on very high-dimensional datasets.

Nevertheless we are curious and we would like to see what the selections algorithm delivers as main features. We will look for the following:  
- PCA: will enable us to visualize the contributions of the pricipal components to the variance and confirm the first point above. 
- T-SNE: alternative non-linear method to see if we have non linear effects.
- Selekt KBest: will tell us if the top 5 features impacting target ar the ones we had anticipated in the EDA.  
- Random forest: this non-linear model will inform us about feature importance, as described in the model selection section. It will also be a robust baseline model, not impacted by outliers of features transformations.  

We will draw conclusions based on those observations, in particular we might conclude that we should extract more of the features types similar to the top5 features (for instance more symbols).

##### Reducing dimensionality to visualize the data projection on 3 principal axes

To look at the source 2 and 3, we will be confronted with high-dimensional datasets potentially difficult to visualize.
The high-dimensional plots are much less intuitive. We will reduce dimension to allow some degree of visualization of the data structure.

Principal Component Analysis (PCA) will be used to choose an “interesting” linear projection of the data that explain a certain % of the variance.
We will make a scree plot to show how many PCA components explain 10%, 20%, …, 90% and 100% of the variance.
We will then plot the data along the first 3 principal components, with additing additional color or size effects to view supplementary dimensions.
We want to observe if the data is spread in linearly separable groups in the defined base.

##### 1. PCA

We first make a scree plot to show how many PCA components explain 10%, 20%, …, 90% and 100% of the variance

In [ ]:
# We setup the libraries for principal component analysis (PCA)

from sklearn.decomposition import PCA

# We also add a standard scaler as well as a pipline object
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
# We create the features matrix and the target vector as numpy arrays

X=df_clean_scrome.drop(['cout_unitaire_pour_la_qte_demandee'], axis=1)\
            .astype(np.float).values

y=df_clean_scrome['cout_unitaire_pour_la_qte_demandee']\
            .astype(np.float).values

# We also register the features names in a list for a later usage
df_clean_scrome_cols=df_clean_scrome\
            .drop(['cout_unitaire_pour_la_qte_demandee'], axis=1)\
            .columns.to_list()

# We register the indexes of our dataframe
X_indexes=df_clean_scrome.index.astype(np.int).values

print('Shape of features vector X: {}'.format(X.shape))
print('Shape of target vector y: {}'.format(y.shape))

In [ ]:
# Create PCA model and fit it to the array of features
n_components=X.shape[1]
pca=make_pipeline(StandardScaler(),PCA(n_components=n_components))
pca.fit(X);

In [ ]:
# Define how many component define a specific percentage of the taget variance

var_cum=np.cumsum(pca.steps[1][1].explained_variance_ratio_)
list_id=[]
for thresh in np.arange(0.10,1.0,0.10):
    list_id.append([i for i in var_cum if i>thresh][0])
for i,j in enumerate(var_cum):
    if j in list_id:
        print('{} component(s) explain {:.0f} % of the variance'\
                .format(1+i,100*j))

In [ ]:
# We plot the corresponding so called 'Scree plot'

plt.figure(figsize=(16,6))
first_n=50
sns.barplot(np.arange(1,n_components+1)[:first_n],100*pca.steps[1][1]\
            .explained_variance_ratio_[:first_n])
plt.step(np.arange(1,n_components+1)[:first_n],
         np.cumsum(100*pca.steps[1][1].explained_variance_ratio_[:first_n]))
plt.xlabel('Principal components')
plt.ylabel('Contribution to total variance in %')
plt.title('Scree plot - % of variance explained by the first {} components'\
          .format(first_n))
plt.show()

We want to investigate how our datapoints are projected into the base made by the 3 principal components, and visualize it in 3D.   
We first need to refit our PCA with n_components=3

In [ ]:
# We now setup the plot of the various datapoint inside newly defined PC base

# We setup libraries for the 3D visualisations with plotly
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px

# We refit the model on 3 components to prepare the base for a display
pca.set_params(pca__n_components=3)
pca.fit(X);

# We project the data onto 3 principal axes
X_3d=pca.transform(X)
df_X_3d=pd.DataFrame(X_3d).set_index(df_clean_scrome.index)
df_X_3d.sort_values(0, ascending=True).head(20)

The projects index 961 seem to haver higher than average first principal component.  
We will see better in the chart the different groups we may have.   
Before we generate the plot, let's analyse how each feature contribute to the 3 principal component defined.

In [ ]:
# We group the results in a dataframe and add the percentage of PCA explained
# by taking the norm of each vector in the PC base

pca0=pca.steps[1][1].components_[0]
pca1=pca.steps[1][1].components_[1]
pca2=pca.steps[1][1].components_[2]

df_results=pd.DataFrame({'variance': X.var(axis=0),
                         'pc1':pca.steps[1][1].components_[0],
                         'pc2':pca.steps[1][1].components_[1],
                         'pc3':pca.steps[1][1].components_[2],
                         'PCA_explained':(pca0**2+pca1**2+pca2**2)**(0.5)})\
                    .set_index(df_clean_scrome.drop(\
                        ['cout_unitaire_pour_la_qte_demandee'], axis=1).columns)

df_results=df_results.sort_values('PCA_explained', ascending=False)

df_results

Analyze each component by inspecting the weights of the loading vectors in the components_ attribute

In [ ]:
# Sort DataFrame by the coefficients of the 1st loading vector

df_results.sort_values('pc1', ascending=False).head(15)

In [ ]:
# Finally plot the target in color in the base made by the 3 first principal components 

fig = plt.figure(figsize=(18,16))
ax = fig.gca(projection='3d')

vectors=1

ax.scatter(X_3d[:,0],
           X_3d[:,1],
           X_3d[:,2],
           zdir='PC 3',
           s=40,
           alpha=0.8,
           marker='o',
           c=y)

ax.quiver(0,
          0,
          0,
          10*df_results[0:vectors].pc1,
          10*df_results[0:vectors].pc2,
          10*df_results[0:vectors].pc3,
          color = 'red',
          alpha = .8,
          lw = 1,)#**{'label':1}

ax.set_xlabel('Principal component 1')
ax.set_ylabel('Principal component 2')
ax.set_zlabel('Principal component 3')

plt.title('PCA plot - % of variance explained')

ax.view_init(elev=25, azim=95)

plt.show()

As an alternative interactive plot we show the same results with the plotly library (this requieres the installation of plotlywidget and jupyterlab plotly extensions)

In [ ]:
# Interactive plot

fig = px.scatter_3d(data_frame=df_X_3d,
                    x=0,
                    y=1,
                    z=2,
                    opacity=0.8,
                    color=y)#size=y[:,0], text=y

fig.update_traces(marker=dict(size=5,
                     line=dict(width=0,
                     color='Black')),
                  selector=dict(mode='markers'))

#fig.write_html('figure.html', auto_open=True)

fig.show()

In this view we can interactively search for possible clusters of datapoints with similar target values. For instance pc0 negative and high components 1 and 2 correspond to projects 532,534,578.  
We can look at those below.

In [ ]:
# Print the list of projects ids available in the test set
# X_indexes

In [ ]:
# Function to display the project technical drawings and informations

from PIL import Image
import os
import glob

project_id=578#X_indexes[10]

print('Project {}:'.format(project_id))
project_id_pdf_path=os.path.normpath(df_merged[df_merged.index==project_id]['pdf_file'].item())
if project_id_pdf_path[-2]=='d': 
    project_id_img_path=project_id_pdf_path[:-4]+'_page1from*.jpg'
else:
    project_id_img_path=project_id_pdf_path[:-5]+'_page1from*.jpg'
#print('{}'.format(project_id_img_path))
jn=glob.glob(os.path.normpath(project_id_img_path))[0]
print(jn)
im=Image.open(jn)
size=800,800
im.thumbnail(size, Image.ANTIALIAS)
display(im)
display(pd.DataFrame(X[list(X_indexes).index(project_id),:], index=df_clean_scrome_cols).head(150))

Indeed those 3 projects seem to have a similar drawing.  
We do not investigate further here for time reasons, but it is interesting to look at points clusters in this kind of chart.  

##### 2. TSNE

To capture potential non linear structures we will also try to use T-SNE, a manifold learning unsupervised algorithm that learns the high-dimensional structure of the data from the data itself, without the use of predetermined classifications.
Manifold learning methods are based on a nearest-neighbor search, so we use the Standard Scaler to unsure our results maningful.

In [ ]:
# We setup the libraries for non-linear dimensionality reduction method T-SNE

# Import necessary libraries
from sklearn.manifold import TSNE

# Build model pipeline including a standard scaler
tsne=make_pipeline(StandardScaler(),TSNE(
    n_components=3,
    perplexity=4.0,
    early_exaggeration=12.0,
    learning_rate=200.0,
    n_iter=5000,
    n_iter_without_progress=300,
    min_grad_norm=1e-07,
    metric='euclidean',
    init='random',
    verbose=0,
    random_state=None,
    method='barnes_hut',
    angle=0.5))

# Fit the model to our arrays of features and generates dataframe
# with projection of data onto 3 pricipal axes
X_3d=tsne.fit_transform(X)
df_X_3d=pd.DataFrame(X_3d)
print(df_X_3d.head())

# Plot target in the base made by the 3 first principal components 
fig = px.scatter_3d(data_frame=df_X_3d, x=0, y=1, z=2, opacity=0.8, color=y)
#size=y[:,0]
fig.update_traces(marker=dict(size=5,
                     line=dict(width=0,
                     color='Black')),
          selector=dict(mode='markers'))
#fig.write_html('figure.html', auto_open=True)
fig.show()

We do not really see clear clusters of points from this experience with t-distributed Stochastic Neighbor Embedding (t-SNE), and we know this method is very sensitive to parameters tuning, so we drop this part of the analysis for the moment.  

If the results is not so clear it's probably because we not no have significant non-linearities in our dataset.  

We could reuse this method again when we work with the source 3 (step files) which contains much larger dimension which may be only artificially large.  
In this case the use of T-SNE method to find non linear structures in the dataset will make more sense.  

##### Reducing dimensionality with selekt k-Best

We usually perform feature selection/dimensionality reduction on sample sets to improve estimators’ accuracy scores or to boost their performance on very high-dimensional datasets. In our case we would like to understand which are the best features based on simple univariate statistical tests.
We use the SelectKBest class which removes all but the highest scoring features
We use the scoring function f_regression as argument in our regression case.

In [ ]:
# Feature selection based on SelectKBest

# Import necessary libraries
from sklearn.feature_selection import SelectKBest,f_regression

# Define number of features to select
k=20

#Select k best features
selector=SelectKBest(f_regression, k=k)
X_new = selector.fit_transform(X, y)
features_select_id=selector.get_support(indices=True)

#Extract the feature names
features_select=[df_clean_scrome_cols[i] for i in features_select_id]
print('Feature selection based on SelectKBest with k={}'.format(k))
features_select

We see that the SelektKbest selects the features we already identified as beeing drivers.

#### Train / Test split

We will perform a train test split just before the model definitions, and keep the test set apart until the end for the scoring of our models on unseen data.  
We will use sklearn.model_selection.train_test_split to perform this split.  

We estimate that a 25% test set to be kept untouched during the modeling phase is a sufficient value as its corresponds to about 70 samples in the test set (with data cardinality of 280 available to date). We note we are short with overall cardinality, but we assume this is sufficiant for this project which is used as a quick demonstrator.  
The risk of this low cardinality in the test set is to have ommitted completely certain categories with low number of representant for instance.  

#### Tuning hyperparameters of the models

The fine tuning of estimator hyperparameters will be done with exhaustive grid search using **GridSearchCV**, which will search for the best cross validated score.  

We will take time to verify that our best hyperparameters are not at the border of the grid parameters range.  

List of **estimators and associated parameters** to be gathered in a parameter grid:  
We note that only the models 2, 3.1 and 3.3 need tuning for the hyperparameters.  
* Model 1: Linear: 0 hypermarameter.
* Model 2: Ridge: 1 hyperparameter: alpha (regularization strength).
* Model 3.1: kNN: 1 hyperparameter: n_neighbors (number of neighbors), weights (contribution of the local neighborhood='uniform').
* Model 3.2: Random Forest: 0 hyperparameter: n_estimator (number of trees in the forest), max_depth (=none), max_features (size of the random subsets of features to consider when splitting a node =none)
* Model 3.3: SVR with rbf kernel: 2 hyperparameters: C (trades off misclassification of training examples against simplicity of the decision surface) and Gamma (how much influence a single training example has).  


**Cross validation strategy**  
* Will be setup with a KFold with a split of 5 to 10. (Or we just assign an interger to the cv argument). 
* 10 will result in a validation set of 40 for a dataset of 400, which is just reasonable to get a meaningful score.

**Score**  
We will use a different metrics for the grid search and for the communication of results, in order to be able to interprete the results.   
* The metrics coefficient of determination R^2 score is the good default parameter used within the GridSearchCV object to defined the ranking of best scores.  
* We will plot the MAE score on the validation curves which is more interpretable for the final comparisons.  

**Complexity**  
The complexity will not be critical since dataset cardinality & dimensionality are rather small on this project.  
Computations will be performed in parallel on the 8 CPU units available on my computer, for the models selected which allow // computation.  
Computational cost as a function of model parameters :
* Ridge, similar to OLS : O(NFeatures\*\*2 \* NSamples).  
* kNN with brute force algorithm: O(NFeatures \* NSamples\*\*2).  
* Random Forest: O(NTrees \* NSamples \* log(NSamples)) with NTrees number of trees in forest. 
* SVR with rbf kernel: from O(NFeatures \* NSamples\*\*2) to O(NFeatures \* NSamples\*\*3).  

**Best scores / best parameters**  
We will display the best parameters and best scores for the models.  
Use of DataFrames to present the mean performance on the training and validation folds and the values tested for each hyperparameter.  
Sort the results according to the average performance on the validation folds.  

**Validation cruves** to visualize over/underfitting  
The validation curves correspond to the performance (MAE scores) on the training and validation folds plotted as as a function of the main hyperparameter.(e.g., using the plt.semilogx() function). We will plot the mean from both validation and train sets with tolerance bands centered on the mean and corresponding to the standard deviations). For the different models we will have:  
* Ridge: MAE function of alpha.  
* kNN: MAE function of neighbors.  
* Random Forest: No need for a validation curve, since there is no concept of over or underfitting as such as there is no hyperparameter impacting complexity (we will not tune the tree depth). The over or underfitting is hidden into the bootstrapping.  
* SVR: MAE function of C. As many plots as Gamma values. (As alternative we will plot a heatmap, for this will will need to check if we can afford for 10 * 10 computations covering 10 points of gamma and 10 points of C, to have a meaningful display.


**Note**  
We will probably select one model which seems more promising and refine the tuning on this model only.  

#### Communicating the results of the hyperparameters tuning & final comparison (calculated on train & validation sets)


MAE will be used since it is easy to interpret in terms of absolute value for our target.

- We will compare MAE metrics :
  - for each of our 4 models:
    - for the best combination of our hyperparameters:
      - we will calculate the mean and standard deviation on the (5 to 10) Kfolds.  

- We note that this validation curve is performed on points which have been used to tune hyperparameters, which is ok since final model evaluation will be done on the test set.

#### Results of the model (calculated on unseen test set) 


- We check that test set is sufficient to peform the analyis: 60 - 80 data points

- MAE score is used for each model for comparison purpose

- For our best model we will make a scatterplot of the predictions against target:  
  - We expect a cloud of points linearly distributed
  - We will observe the shape of this cloud and make conclusion on the efficiency of our model for certain values of the target (for instance large or low values)
  - We will investigate further the predicted values repartition by adding hue information of some categories of quantitative features, this will help us to see if the model performs +- better in some specific case, and eventually draw conclusion on our features selection/extraction/preprocessing. We already think about the indicator of low quality drawings as a candidate for this test, we will also do it with the top 5 main features selected by our RandomForest most important features.
  - We can also check how the errors are correlated to the features with a correlation coefficient.

In [ ]:
df_clean_scrome.drop(['cout_unitaire_pour_la_qte_demandee'], axis=1)

In [ ]:
# Import the necessary libraries

from sklearn.model_selection import train_test_split

print('Shape of features matrix X: {}'.format(X.shape))
print('Shape of target vector y: {}'.format(y.shape))

# Test train split
# Note that in this case we also register the index for a later usage

test_size=0.25
idx=0
X_tr,X_te,y_tr,y_te,X_tr_index,X_te_index=train_test_split(X,y,X_indexes,
                                                           test_size=test_size,
                                                           random_state=idx)

print('Shape train features matrix X_tr:{}'.format(X_tr.shape))
print('Shape train target vector y_tr:{}'.format(y_tr.shape))
print('Shape test features matrix X_te:{}'.format(X_te.shape))
print('Shape test target vector y_te:{}'.format(y_te.shape))
print('Shape train features matrix index X_tr_index:{}'\
      .format(X_tr_index.shape))
print('Shape test features matrix index X_tr_index:{}'\
      .format(X_te_index.shape))

# We mirror this split into original dataframes for later use in this notebook
df_clean_scrome_train=df_clean_scrome[df_clean_scrome.index.isin(X_tr_index)].copy()
df_clean_scrome_test=df_clean_scrome[df_clean_scrome.index.isin(X_te_index)].copy()

In [ ]:
# Optional addition of test elements from other clients
# We create the features matrix and the target vector as numpy arrays

X_o=df_clean_other.drop(['cout_unitaire_pour_la_qte_demandee'], axis=1)\
            .astype(np.float).values

y_o=df_clean_other['cout_unitaire_pour_la_qte_demandee']\
            .astype(np.float).values

# We also register the features names in a list for a later usage
df_clean_other_cols=df_clean_other\
            .drop(['cout_unitaire_pour_la_qte_demandee'], axis=1)\
            .columns.to_list()

# We register the indexes of our dataframe
X_o_index=df_clean_other.index.astype(np.int).values

print('Shape of features vector X_o: {}'.format(X_o.shape))
print('Shape of target vector y_o: {}'.format(y_o.shape))

# We label original dataframes similar to the scrome dataset
df_clean_other_test=df_clean_other.copy()

In the next sections we will perform the modelisation and therefore work only with train set. 

#### Metrics

We will report the MAE metric computed on the target values in euros (without log scaling) which is easier to interprete.  
The coefficient of determinition R^2 is also calculated and reported, we use it since this is a good default parameter in GridSearchCV).  

#### Model 0 - Baseline

We will apply the model on all variables.  
We use a dummy regressor with a 'mean' strategy.  
We generate prediction and compute the mean absolute error on the complete dataset.  

In [ ]:
# Model 0 - Baseline
# This first model is based on the 'cout_unitaire_pour_la_qte_demandee' mean. 

# Import the necessary libraries
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error as MAE, r2_score

# Create and fit model
model_0=DummyRegressor(strategy='mean')
model_0.fit(X_tr,y_tr)
        
# Generate predictions    
y_tr_pred=model_0.predict(X_tr)
y_te_pred=model_0.predict(X_te)
        
# Get performance metrics
# For the MAE we convert the target back to its original scale
MAE_tr_model_0=MAE(10**y_tr,10**y_tr_pred)
R2_tr_model_0=r2_score(y_tr,y_tr_pred)
MAE_te_model_0=MAE(10**y_te,10**y_te_pred)
R2_te_model_0=r2_score(y_te,y_te_pred)

#Gather the predictions of the data and print results

#TRAIN SET
prediction_model_0_train_df=pd.DataFrame()
prediction_model_0_train_df['id']=X_tr_index
prediction_model_0_train_df['cout_unitaire_pour_la_qte_demandee']=np.round(10**y_tr_pred, decimals=1)

print('Baseline model 0 MAE (train & valid set): {:.1f}'.format(MAE_tr_model_0))
print('Baseline model 0 R2 (train & valid set): {:.3f}'.format(R2_tr_model_0))
display(prediction_model_0_train_df.head())

#TEST SET
prediction_model_0_test_df=pd.DataFrame()
prediction_model_0_test_df['id']=X_te_index
prediction_model_0_test_df['cout_unitaire_pour_la_qte_demandee']=np.round(10**y_te_pred, decimals=1)

print('Baseline model 0 MAE (test set): {:.1f}'.format(MAE_te_model_0))
print('Baseline model 0 R2 (test set): {:.3f}'.format(R2_te_model_0))
display(prediction_model_0_test_df.head())

The predictions are all equal to the target mean in this model as per definition.  
Coefficient of determination R2 is zero per definition.  

#### Model 1 - Linear regression model with 'human' selected variables

We will apply the model on 3 variables, which seem to be slightly more correlated with our target from our preliminary EDA: 
- 'chiffrage_pour_la_qte_demandee_quantite_unite'
- '±'
- 'quality_indicator_kurtosis'

The pipeline will include:
- Standard Scaler to help the optimisation algorithm converge.  
- Estimator: Linear regression

In this first model we use the GridSearchCV object to perform the cross-validation only.  
We use it keeping the argument param_grid empty as there are no parameter to tune.  

In [ ]:
# Model 1 - Linear regression model with 'human' selected variables

# We want to select only a few columns corresponding to the features we would like to adress in our features matrix
# To do this we set a function to manage correspondance between dataframe columns and np.array corresponding columns indexes
def id(L,c):
    Li=[]
    for i in c:
        Li.append(L.index(i))
    return Li
feature_set_model_1=['chiffrage_pour_la_qte_demandee_quantite_unite','±','quality_indicator_kurtosis']
feature_set_model_1_id=id(df_clean_scrome_cols,feature_set_model_1)

# Import the necessary libraries
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, KFold #ShuffleSplit

# Create model
model_1=make_pipeline(StandardScaler(),LinearRegression())

# Param grid definition for gridsearch is empty since we have no hyperparameter to tune
param_grid_1=[{}]

# Cross validation strategy
cv_1=KFold(n_splits=10, shuffle=True, random_state=0)
#cv=(n_splits=10, test_size=0.3, random_state=0)

# Grid search
gs_1=GridSearchCV(estimator=model_1,
                  param_grid=param_grid_1,
                  n_jobs=-1,
                  refit=True,
                  cv=cv_1,
                  verbose=1,
                  return_train_score=True)

In [ ]:
# We fit model on the train set
gs_1.fit(X_tr[:,feature_set_model_1_id],y_tr)

# We display the best parameters for the model
# We note there are no hyperparameter in this model
print('Best parameters model 1: {}'.format(gs_1.best_params_))

# We display the model corresponding best score
# We note there are no hyperparameter in this model
print('Score model 1 (train & valid set): {:.2f}'.format(gs_1.score(X_tr[:,feature_set_model_1_id],y_tr)))
print('Score model 1 (test set): {:.2f}'.format(gs_1.score(X_te[:,feature_set_model_1_id],y_te)))

In [ ]:
# Generate predictions
# We note there are no hyperparameter in this model
y_tr_pred=gs_1.predict(X_tr[:,feature_set_model_1_id])
y_te_pred=gs_1.predict(X_te[:,feature_set_model_1_id])
        
# Get performance metrics
# For the MAE we convert the target back to its original scale
MAE_tr_model_1=MAE(10**y_tr,10**y_tr_pred)
R2_tr_model_1=r2_score(y_tr,y_tr_pred)
MAE_te_model_1=MAE(10**y_te,10**y_te_pred)
R2_te_model_1=r2_score(y_te,y_te_pred)

#Gather the predictions of the data and print results

#TRAIN SET
prediction_model_1_train_df=pd.DataFrame()
prediction_model_1_train_df['id']=X_tr_index
prediction_model_1_train_df['cout_unitaire_pour_la_qte_demandee']=np.round(10**y_tr_pred, decimals=1)

print('Model 1 MAE (train & valid set): {:.1f}'.format(MAE_tr_model_1))
print('Model 1 R2 (train & valid set): {:.3f}'.format(R2_tr_model_1))
display(prediction_model_1_train_df.head())

#TEST SET
prediction_model_1_test_df=pd.DataFrame()
prediction_model_1_test_df['id']=X_te_index
prediction_model_1_test_df['cout_unitaire_pour_la_qte_demandee']=np.round(10**y_te_pred, decimals=1)

print('Model 1 MAE (test set): {:.1f}'.format(MAE_te_model_1))
print('Model 1 R2 (test set): {:.3f}'.format(R2_te_model_1))
display(prediction_model_1_test_df.head())

In [ ]:
cv_results_cols_drop=['mean_fit_time','std_fit_time','mean_score_time','std_score_time',
                                    'params','split0_test_score','split1_test_score','split2_test_score',
                                    'split3_test_score','split4_test_score','split5_test_score',
                                    'split6_test_score','split0_train_score','split1_train_score',
                                    'split2_train_score','split3_train_score','split4_train_score',
                                    'split5_train_score','split6_train_score']

cv_1_results = pd.DataFrame(gs_1.cv_results_).sort_values('rank_test_score')
cv_results_cols=cv_1_results.drop(cv_results_cols_drop, axis=1).columns
cv_1_results=cv_1_results[cv_results_cols].astype(float).copy()
cv_1_results.head()

With this preliminary simple linear regression model without regularisation, we can predict our target using 3 important features preselected. The model converges towards a solution with a MAE error which is lower than our baseline. This is encouraging. We will now try to incorporate more features. For this we will use a Ridge model with regularisation.

#### Model 2 - Ridge linear regression model with all variables

We will apply the model on the complete features space.  

The pipeline will include:  
- OPTIONAL depending on results: transforms on the floats extracted from drawings (\*\*2, \*\*3, log, \*\*0.5).  
- Addition of polynomial features on the floats extracted from drawings.  
- Standard Scaler to help the optimisation algorithm converge and avoid ill-conditionning.  
- Estimator: Ridge regression. 

In [ ]:
# Definition of model 2: linear regression model with ridge estimator

# Import necessary libraries
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn import set_config
set_config(display='diagram')

# Set up columns transformer to include polynomial features

poly_col_id=id(df_clean_scrome_cols,['tolerance_band_mean',
                              'tolerance_band_median',
                              'tolerance_band_quantile_20',
                              'tolerance_band_quantile_80',
                              'dimension_band_mean',
                              'dimension_band_median',
                              'dimension_band_quantile_20',
                              'dimension_band_quantile_80'])
# Currently not used:
#'±','Ra','Ø',
#'tolerance_band_count',
#'dimension_band_count',
#'tolerance_band_min',
#'tolerance_band_max',
#'tolerance_band_std',     
#'tolerance_band_irq',
#'dimension_band_min',
#'dimension_band_max',
#'dimension_band_std',
#'dimension_band_irq',
#'quality_indicator_kurtosis'
# As been tried out and generates overfitting: Score on the train is better but lower on the test set.
# Probably we could use those poly features when we have more data points.
# When setting the only parameters selected above with degree 2: we get slightly better scores.


print('columns for addition of polynomial features:', poly_col_id)
poly_transformer = make_pipeline(PolynomialFeatures(degree=2, include_bias=False))
transformer = ColumnTransformer([('poly',poly_transformer,poly_col_id)],remainder='passthrough')

# Pipeline definition
model_2=Pipeline([('transformer',transformer),#None
                 ('scaler',StandardScaler()),
                 ('estimator',Ridge(alpha=1.0,
                                    fit_intercept=True,
                                    normalize=False,
                                    copy_X=True,
                                    max_iter=None,
                                    tol=0.001,
                                    solver='auto',
                                    random_state=0))])

# Param grid definition for gridsearch
param_grid_2=[{'estimator__alpha':np.logspace(-6,6,num=50)}]

# Cross validation strategy
cv_2=KFold(n_splits=10, shuffle=True, random_state=0)
#cv=(n_splits=10, test_size=0.3, random_state=0)

# Definition of a custom scorer to be able to score the target values without log scale
from sklearn.metrics import make_scorer
def mae_func(y,y_pred):
    return -np.mean(np.abs(10**y-10**y_pred))
custom_scorer=make_scorer(mae_func,greater_is_better=False)


# Grid search
gs_2=GridSearchCV(estimator=model_2,
                  param_grid=param_grid_2,
                  scoring={'R2':'r2','MAE':custom_scorer},#None,neg_mean_absolute_error
                  n_jobs=-1,
                  refit='R2',#True,
                  cv=cv_2,
                  verbose=1,
                  return_train_score=True)

In [ ]:
# We fit the model on the train set
gs_2.fit(X_tr,y_tr)

# We display the best parameters for the model
print('Best parameters model 2: {}'.format(gs_2.best_params_))

# We display the model corresponding best score
print('Score model 2 (train & valid set): {:.2f}'.format(gs_2.score(X_tr,y_tr)))
print('Score model 2 (test set): {:.2f}'.format(gs_2.score(X_te,y_te)))

In [ ]:
# Generate predictions
# We note the model has been automatically refit on the best parameters
y_tr_pred=gs_2.predict(X_tr)
y_te_pred=gs_2.predict(X_te)
y_o_pred=gs_2.predict(X_o)

# Get performance metrics
# For the MAE we convert the target back to its original scale
MAE_tr_model_2=MAE(10**y_tr,10**y_tr_pred)
R2_tr_model_2=r2_score(y_tr,y_tr_pred)
MAE_te_model_2=MAE(10**y_te,10**y_te_pred)
R2_te_model_2=r2_score(y_te,y_te_pred)

MAE_o_model_2=MAE(10**y_o,10**y_o_pred)
R2_o_model_2=r2_score(y_o,y_o_pred)

#Gather the predictions of the data and print results

#TRAIN SET
prediction_model_2_train_df=pd.DataFrame()
prediction_model_2_train_df['id']=X_tr_index
prediction_model_2_train_df['cout_unitaire_pour_la_qte_demandee']=np.round(10**y_tr_pred, decimals=1)

print('Model 2 MAE (train & valid set): {:.1f}'.format(MAE_tr_model_2))
print('Model 2 R2 (train & valid set): {:.3f}'.format(R2_tr_model_2))
display(prediction_model_2_train_df.head())

#TEST SET
prediction_model_2_test_df=pd.DataFrame()
prediction_model_2_test_df['id']=X_te_index
prediction_model_2_test_df['cout_unitaire_pour_la_qte_demandee']=np.round(10**y_te_pred, decimals=1)

print('Model 2 MAE (test set): {:.1f}'.format(MAE_te_model_2))
print('Model 2 R2 (test set): {:.3f}'.format(R2_te_model_2))
display(prediction_model_2_test_df.head())

#TEST SET OTHER
prediction_model_2_other_df=pd.DataFrame()
prediction_model_2_other_df['id']=X_o_index
prediction_model_2_other_df['cout_unitaire_pour_la_qte_demandee']=np.round(10**y_o_pred, decimals=1)

print('Model 2 MAE (other set): {:.1f}'.format(MAE_o_model_2))
print('Model 2 R2 (other set): {:.3f}'.format(R2_o_model_2))
display(prediction_model_2_other_df.head())

In [ ]:
#Definition of the model results and model validation display function

# Extract the GridSearchCV results
cv_2_results = pd.DataFrame(gs_2.cv_results_)\
                    .drop('params', axis=1)\
                    .astype(float)\
                    .sort_values('param_estimator__alpha')
display(cv_2_results.head())

# Name the variables to plot 
alphas=np.log10(cv_2_results['param_estimator__alpha'])

rank_te_R2_2=cv_2_results['rank_test_R2']
mean_te_R2_2=cv_2_results['mean_test_R2']
mean_tr_R2_2=cv_2_results['mean_train_R2']
std_te_R2_2=cv_2_results['std_test_R2']
std_tr_R2_2=cv_2_results['std_train_R2']

rank_te_MAE_2=cv_2_results['rank_test_MAE']
mean_te_MAE_2=cv_2_results['mean_test_MAE']
mean_tr_MAE_2=cv_2_results['mean_train_MAE']
std_te_MAE_2=cv_2_results['std_test_MAE']
std_tr_MAE_2=cv_2_results['std_train_MAE']

# Identify the GridSearchCV best value
idxmin_2=rank_te_R2_2.idxmin()
alpha_idxmin_2=alphas[idxmin_2]
rank_te_R2_idxmin_2=rank_te_R2_2[idxmin_2]
mean_te_R2_idxmin_2=mean_te_R2_2[idxmin_2]
std_te_R2_idxmin_2=std_te_R2_2[idxmin_2]
rank_te_MAE_idxmin_2=rank_te_MAE_2[idxmin_2]
mean_te_MAE_idxmin_2=mean_te_MAE_2[idxmin_2]
std_te_MAE_idxmin_2=std_te_MAE_2[idxmin_2]

# Store the MAE score for each CV run for final comparison plot
S2=cv_2_results.loc[idxmin_2,[c for c in cv_2_results.columns if ('_test_MAE' in c)&('split' in c)]]
S2

In [ ]:
# Plot mean scores
plt.figure(figsize=(8,5))
plt.plot(alphas,mean_te_MAE_2, label='validation')
plt.plot(alphas,mean_tr_MAE_2, label='train')

# Quantify variance of those scores with ±std curves
plt.fill_between(alphas,
                 mean_te_MAE_2-std_te_MAE_2,
                 mean_te_MAE_2+std_te_MAE_2,
                 alpha=0.2)
plt.fill_between(alphas,
                 mean_tr_MAE_2-std_tr_MAE_2,
                 mean_tr_MAE_2+std_tr_MAE_2,
                 alpha=0.2)

# Add marker for best score
plt.scatter(alpha_idxmin_2, mean_te_MAE_idxmin_2, marker='o', c='green', zorder=5)

# Print best scores
plt.title('Best alpha: {:.4f}\nR2 score: {:.3f} \u00b1 {:.3f} (rank {:.0f})'\
          .format(10**alpha_idxmin_2,mean_te_R2_idxmin_2,std_te_R2_idxmin_2,rank_te_R2_idxmin_2),
          loc='left')
plt.annotate('MAE: {:.1f} \u00b1 {:.1f}'.format(mean_te_MAE_idxmin_2,std_te_MAE_idxmin_2),
             xy=(alpha_idxmin_2, mean_te_MAE_idxmin_2),
             xytext=(1.8*alpha_idxmin_2, 0.75*mean_te_MAE_idxmin_2),
             arrowprops=dict(facecolor='green', shrink=0.05))
plt.xlabel('$log_{10}(alpha)$')
plt.ylabel('MAE')
plt.legend(loc=2)
plt.grid(color='black', linestyle='--', linewidth=0.1)
plt.show()

The ridge model is fitted and the hyperparameter alpha optimized to find an optimal MAE score. The MAE error is lower than our model 1. This is encouraging. 
We will look later in the ML section how the predictions in terms of MAE translates for high or low values of our target.  
For the moment the model is not performing well enough to be really usable.  
We will now try to incorporate non linearity in the model.  

#### Model 3.1 - kNN non-linear model with all variables

We will apply the model on the complete features space. 

The pipeline will include:  
- Standard Scaler to help the optimisation algorithm converge and avoid ill-conditionning.  
- Estimator: kNN

In [ ]:
# Definition of model 31: non-linear model with kNN estimator

# Import necessary libraries
from sklearn.neighbors import KNeighborsRegressor

# Set up columns transformer to include polynomial features
# Currently not used
poly_col_id=id(df_clean_scrome_cols,[])
print('columns for addition of polynomial features:', poly_col_id)
poly_transformer = make_pipeline(PolynomialFeatures(degree=3, include_bias=False))
transformer = ColumnTransformer([('poly',poly_transformer,poly_col_id)],remainder='passthrough')

# Pipeline definition
model_31=Pipeline([('transformer',None),#transformer
                 ('scaler',StandardScaler()),
                 ('estimator',KNeighborsRegressor(n_neighbors=5,
                                                  weights='uniform',
                                                  algorithm='auto',
                                                  leaf_size=30,
                                                  p=2,
                                                  metric='minkowski',
                                                  metric_params=None,
                                                  n_jobs=-1))])

# Param grid definition for gridsearch
param_grid_31=[{'estimator__n_neighbors':[1,2,3,4,5,6,7,8,9,10]}]

# Cross validation strategy
cv_31=KFold(n_splits=10, shuffle=True, random_state=0)
#cv=(n_splits=10, test_size=0.3, random_state=0)

# Grid search
gs_31=GridSearchCV(estimator=model_31,
                   param_grid=param_grid_31,
                   scoring={'R2':'r2','MAE':custom_scorer},#None
                   n_jobs=-1,
                   refit='R2',#True,
                   cv=cv_31,
                   verbose=1,
                   return_train_score=True)

In [ ]:
# We fit the model to the train set
gs_31.fit(X_tr,y_tr)

# We display the best parameters for the model
print('Best parameters model 31: {}'.format(gs_31.best_params_))

# We display the model corresponding best score
print('Score model 31 (train & valid sets): {:.2f}'.format(gs_31.score(X_tr,y_tr)))
print('Score model 31 (test set): {:.2f}'.format(gs_31.score(X_te,y_te)))

In [ ]:
# Generate predictions
# We note the model has been automatically refit on the best parameters
y_tr_pred=gs_31.predict(X_tr)
y_te_pred=gs_31.predict(X_te)
        
# Get performance metrics
# For the MAE we convert the target back to its original scale
MAE_tr_model_31=MAE(10**y_tr,10**y_tr_pred)
R2_tr_model_31=r2_score(y_tr,y_tr_pred)
MAE_te_model_31=MAE(10**y_te,10**y_te_pred)
R2_te_model_31=r2_score(y_te,y_te_pred)

#Gather the predictions of the data and print results

#TRAIN SET
prediction_model_31_train_df=pd.DataFrame()
prediction_model_31_train_df['id']=X_tr_index
prediction_model_31_train_df['cout_unitaire_pour_la_qte_demandee']=np.round(10**y_tr_pred, decimals=1)

print('Model 31 MAE (train & valid set): {:.1f}'.format(MAE_tr_model_31))
print('Model 31 R2 (train & valid set): {:.3f}'.format(R2_tr_model_31))
display(prediction_model_31_train_df.head())

#TEST SET
prediction_model_31_test_df=pd.DataFrame()
prediction_model_31_test_df['id']=X_te_index
prediction_model_31_test_df['cout_unitaire_pour_la_qte_demandee']=np.round(10**y_te_pred, decimals=1)

print('Model 31 MAE (test set): {:.1f}'.format(MAE_te_model_31))
print('Model 31 R2 (test set): {:.3f}'.format(R2_te_model_31))
display(prediction_model_31_test_df.head())

In [ ]:
#Definition of the model results and model validation display function

# Extract the GridSearchCV results
cv_31_results = pd.DataFrame(gs_31.cv_results_)\
                    .drop('params', axis=1)\
                    .astype(float)\
                    .sort_values('param_estimator__n_neighbors')
display(cv_31_results.head())

# Name the variables to plot 
nneighs=cv_31_results['param_estimator__n_neighbors']

rank_te_R2_31=cv_31_results['rank_test_R2']
mean_te_R2_31=cv_31_results['mean_test_R2']
mean_tr_R2_31=cv_31_results['mean_train_R2']
std_te_R2_31=cv_31_results['std_test_R2']
std_tr_R2_31=cv_31_results['std_train_R2']

rank_te_MAE_31=cv_31_results['rank_test_MAE']
mean_te_MAE_31=cv_31_results['mean_test_MAE']
mean_tr_MAE_31=cv_31_results['mean_train_MAE']
std_te_MAE_31=cv_31_results['std_test_MAE']
std_tr_MAE_31=cv_31_results['std_train_MAE']

# Identify the GridSearchCV best value
idxmin_31=rank_te_R2_31.idxmin()
nneigh_idxmin_31=nneighs[idxmin_31]
rank_te_R2_idxmin_31=rank_te_R2_31[idxmin_31]
mean_te_R2_idxmin_31=mean_te_R2_31[idxmin_31]
std_te_R2_idxmin_31=std_te_R2_31[idxmin_31]
rank_te_MAE_idxmin_31=rank_te_MAE_31[idxmin_31]
mean_te_MAE_idxmin_31=mean_te_MAE_31[idxmin_31]
std_te_MAE_idxmin_31=std_te_MAE_31[idxmin_31]

# Store the MAE score for each CV run for final comparison plot
S31=cv_31_results.loc[idxmin_31,[c for c in cv_31_results.columns if ('_test_MAE' in c)&('split' in c)]]
S31

In [ ]:
# Plot mean scores
plt.figure(figsize=(8,5))
plt.plot(nneighs,mean_te_MAE_31, label='validation')
plt.plot(nneighs,mean_tr_MAE_31, label='train')

# Quantify variance of those scores with ±std curves
plt.fill_between(nneighs,
                 mean_te_MAE_31-std_te_MAE_31,
                 mean_te_MAE_31+std_te_MAE_31,
                 alpha=0.2)
plt.fill_between(nneighs,
                 mean_tr_MAE_31-std_tr_MAE_31,
                 mean_tr_MAE_31+std_tr_MAE_31,
                 alpha=0.2)

# Add marker for best score
plt.scatter(nneigh_idxmin_31, mean_te_MAE_idxmin_31, marker='o', c='green', zorder=5)

# Print best scores
plt.title('Best n_neighbors: {:.4f}\nR2 score: {:.3f} \u00b1 {:.3f} (rank {:.0f})'\
          .format(nneigh_idxmin_31,mean_te_R2_idxmin_31,std_te_R2_idxmin_31,rank_te_R2_idxmin_31),
          loc='left')
plt.annotate('MAE: {:.1f} \u00b1 {:.1f}'.format(mean_te_MAE_idxmin_31,std_te_MAE_idxmin_31),
             xy=(nneigh_idxmin_31, mean_te_MAE_idxmin_31),
             xytext=(1.8*nneigh_idxmin_31, 0.75*mean_te_MAE_idxmin_31),
             arrowprops=dict(facecolor='green', shrink=0.05))
plt.xlabel('n_neighbors')
plt.ylabel('MAE')
plt.legend(loc=2)
plt.grid(color='black', linestyle='--', linewidth=0.1)
plt.show()

The kNN model is fitted and the hyperparameter n_neighbors optimized to find an optimal MAE score. The MAE error is lower than our model 2. This is encouraging. We nevertheless get a MAE score which is not sufficient for our application. 

One usefull characteristic is that we can try to visualize the nearest neigbors of one project.

In [ ]:
# Print the list of projects ids available in the test set
X_te_index

In [ ]:
#We define a fonction to extract the first neighbors of a project in the test set, and a one to plot them

from PIL import Image
import os
import glob

# Function to find the neighbors of a project by project from its id

def neighbors(proj_id):
    N=gs_31.best_estimator_\
                .steps[2][1]\
                .kneighbors(X=X_te[list(X_te_index)\
                                   .index(proj_id),:]\
                            .reshape((1,-1)),
                            n_neighbors=n_nearest,
                            return_distance=True)
    
    return list(N[1][0]), list(N[0][0])

n_nearest=3

project_id=X_te_index[10]

print('The first {} neighbors of the project id {} are: {}\n'\
      .format(n_nearest,
              project_id,
              neighbors(project_id)[0]))

# Function to display the project and corresponding neighbors technical
# drawings and informations

# Display project drawing and informations

print('Project {}:'.format(project_id))
project_id_pdf_path=os.path.normpath(df_merged[df_merged.index==project_id]['pdf_file'].item())
if project_id_pdf_path[-2]=='d': 
    project_id_img_path=project_id_pdf_path[:-4]+'_page1from*.jpg'
else:
    project_id_img_path=project_id_pdf_path[:-5]+'_page1from*.jpg'
#print('{}'.format(project_id_img_path))
jn=glob.glob(os.path.normpath(project_id_img_path))[0]
print(jn)
im=Image.open(jn)
size=400,400
im.thumbnail(size, Image.ANTIALIAS)
display(im)
display(pd.DataFrame(X_te[list(X_te_index).index(project_id),:], index=df_clean_scrome_cols))

# Display neighbors drawings and informations

print('\n{} nearest neighbors:'.format(n_nearest))
list_pdf=list(df_merged[df_merged.index.isin(neighbors(project_id)[0])]['pdf_file'])
list_images=[i[:-4]+'_page1from*.jpg' if i[-2]=='d' else i[:-5]+'_page1from*.jpg' for i in list_pdf]
for i in list_images:
    #print('{}'.format(os.path.normpath(i)))
    jn=glob.glob(os.path.normpath(i))[0]
    print(jn)
    im=Image.open(jn)
    size=400,400
    im.thumbnail(size, Image.ANTIALIAS)
    display(im)

In this exemple we see that the 3 neighbors include 2 times the same drawing; this is because th drawing is not the only parameter in our features space, and we can have several project with same drawing and different quantities for instance.  

#### Model 3.2 - Random Forest non-linear model with all variables

We will apply the model on the complete features space. 

The pipeline will include:   
- No scaler, no feature engineering.  
- Estimator: Random Forest. 

In [ ]:
# Definition of model 32: non-linear model with Random Forest estimator

# Import the necessary libraries
from sklearn.ensemble import RandomForestRegressor

# Create and fit model
model_32=RandomForestRegressor(n_estimators=1000,
                               criterion='mse',
                               max_depth=None,
                               min_samples_split=2,
                               min_samples_leaf=1,
                               min_weight_fraction_leaf=0.0,
                               max_features='auto',
                               max_leaf_nodes=None,
                               min_impurity_decrease=0.0,
                               min_impurity_split=None,
                               bootstrap=True,
                               oob_score=False,
                               n_jobs=-1,
                               random_state=0,
                               verbose=1,
                               warm_start=False,
                               ccp_alpha=0.0,
                               max_samples=None)

# We fit the model on the train set
model_32.fit(X_tr,y_tr)
        
# Generate predictions    
y_tr_pred=model_32.predict(X_tr)
y_te_pred=model_32.predict(X_te)
        
# Get performance metrics
# For the MAE we convert the target back to its original scale
MAE_tr_model_32=MAE(10**y_tr,10**y_tr_pred)
R2_tr_model_32=r2_score(y_tr,y_tr_pred)
MAE_te_model_32=MAE(10**y_te,10**y_te_pred)
R2_te_model_32=r2_score(y_te,y_te_pred)

#Gather the predictions of the data and print results

#TRAIN SET
prediction_model_32_train_df=pd.DataFrame()
prediction_model_32_train_df['id']=X_tr_index
prediction_model_32_train_df['cout_unitaire_pour_la_qte_demandee']=np.round(10**y_tr_pred, decimals=1)

print('Model 32 MAE (train & valid set): {:.1f}'.format(MAE_tr_model_32))
print('Model 32 R2 (train & valid set): {:.3f}'.format(R2_tr_model_32))
display(prediction_model_32_train_df.head())

#TEST SET
prediction_model_32_test_df=pd.DataFrame()
prediction_model_32_test_df['id']=X_te_index
prediction_model_32_test_df['cout_unitaire_pour_la_qte_demandee']=np.round(10**y_te_pred, decimals=1)

print('Model 32 MAE (test set): {:.1f}'.format(MAE_te_model_32))
print('Model 32 R2 (test set): {:.3f}'.format(R2_te_model_32))
display(prediction_model_32_test_df.head())

In [ ]:
# Checking the feature importance
# (Code adapted from the sklearn library exemple)

importances = model_32.feature_importances_

std = np.std([tree.feature_importances_ for tree in model_32.estimators_],axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
n_first=20
print('Feature ranking ({} first):'.format(n_first))

feature_names=[]
for f in range(X.shape[1])[:n_first]:
    print('{:>6}. feature {:>3} ({:.6f}) : {}'.format(f + 1, indices[f], importances[indices[f]],df_clean_scrome_cols[indices[f]]))
    feature_names.append(df_clean_scrome_cols[indices[f]])

# Plot the impurity-based feature importances of the forest
fig, ax = plt.subplots(figsize=(18,8))
plt.title('Feature importances (first {}): impurity-based feature importances of the forest, along with their inter-trees variability'.format(n_first))
ax.barh(range(X.shape[1])[:n_first], importances[indices][:n_first],
        color='r', xerr=std[indices][:n_first], align='center')
plt.yticks(range(X.shape[1])[:n_first], feature_names,rotation=0)
#plt.xlim([-1, X.shape[1]])
ax.invert_yaxis()
plt.show()

Random forest appear to be our best model in terms of MAE score.  
We are also interested in the ranking of features importance as we have announced before.  
This is here extremely interesting as it confirms our previous findings about the importance of features, in particular it shows that the ± extracted parameter is part of the main drivers. It tells us that we should try to extract more of those from our drawings, similar ones, which should help our model to give better results.  

#### Model 3.3 - SVR non-linear model with all variables

We will apply the model on the complete features space. 

The pipeline will include:  

- No feature enginnering (to be checked).
- Standard Scaler to help the optimisation algorithm converge and avoid ill-conditionning.  
- Estimator: SVR with rbf kernel.

In [ ]:
# Definition of model 33: non-linear model with SVR estimator

# Import necessary libraries
from sklearn.svm import SVR

# Set up columns transformer to include polynomial features
# Currently not used
poly_col_id=id(df_clean_scrome_cols,[])
print('columns for addition of polynomial features:', poly_col_id)
poly_transformer = make_pipeline(PolynomialFeatures(degree=3, include_bias=False))
transformer = ColumnTransformer([('poly',poly_transformer,poly_col_id)],remainder='passthrough')

# Pipeline definition
model_33=Pipeline([('transformer',None),#transformer
                 ('scaler',StandardScaler()),
                 ('estimator',SVR(kernel='rbf',
                                  gamma='scale',
                                  tol=0.001,
                                  C=1.0,
                                  epsilon=0.1,
                                  shrinking=True,
                                  cache_size=200,
                                  verbose=True,
                                  max_iter=-1,))])

# Param grid definition for gridsearch
param_grid_33=[{'estimator__C':np.logspace(-4,7,num=55),
                'estimator__epsilon':np.logspace(-6,0,num=25)}]

# Cross validation strategy
cv_33=KFold(n_splits=10, shuffle=True, random_state=0)
#cv=(n_splits=10, test_size=0.3, random_state=0)

# Grid search
gs_33=GridSearchCV(estimator=model_33,
                   param_grid=param_grid_33,
                   scoring={'R2':'r2','MAE':custom_scorer},#custom_scorer,
                   n_jobs=-1,
                   refit='R2',#True,
                   cv=cv_33,
                   verbose=1,
                   return_train_score=True)

In [ ]:
# We fit the model to the train set
gs_33.fit(X_tr,y_tr)

# We display the best parameters for the model
print('Best parameters model 33: {}'.format(gs_33.best_params_))

# We display the model corresponding best score
print('Score model 33 (train & valid set): {:.2f}'.format(gs_33.score(X_tr,y_tr)))
print('Score model 33 (test set): {:.2f}'.format(gs_33.score(X_te,y_te)))

In [ ]:
# Generate predictions
# We note the model has been automatically refit on the best parameters
y_tr_pred=gs_33.predict(X_tr)
y_te_pred=gs_33.predict(X_te)
        
# Get performance metrics
# For the MAE we convert the target back to its original scale
MAE_tr_model_33=MAE(10**y_tr,10**y_tr_pred)
R2_tr_model_33=r2_score(y_tr,y_tr_pred)
MAE_te_model_33=MAE(10**y_te,10**y_te_pred)
R2_te_model_33=r2_score(y_te,y_te_pred)

#Gather the predictions of the data and print results

#TRAIN SET
prediction_model_33_train_df=pd.DataFrame()
prediction_model_33_train_df['id']=X_tr_index
prediction_model_33_train_df['cout_unitaire_pour_la_qte_demandee']=np.round(10**y_tr_pred, decimals=1)

print('Model 33 MAE (train & valid set): {:.1f}'.format(MAE_tr_model_33))
print('Model 33 R2 (train & valid set): {:.3f}'.format(R2_tr_model_33))
display(prediction_model_33_train_df.head())

#TEST SET
prediction_model_33_test_df=pd.DataFrame()
prediction_model_33_test_df['id']=X_te_index
prediction_model_33_test_df['cout_unitaire_pour_la_qte_demandee']=np.round(10**y_te_pred, decimals=1)

print('Model 33 MAE (test set): {:.1f}'.format(MAE_te_model_33))
print('Model 33 R2 (test set): {:.3f}'.format(R2_te_model_33))
display(prediction_model_33_test_df.head())

In [ ]:
# Definition of the model results and model validation display function

# Extract the GridSearchCV results
cv_33_results = pd.DataFrame(gs_33.cv_results_)\
                    .drop('params', axis=1)\
                    .astype(float)\
                    .sort_values('param_estimator__C')

for epsilon in cv_33_results['param_estimator__epsilon'].sort_values().unique().tolist():

    # Filter the data frame by epsilon
    cv_33_results_epsilon=cv_33_results[cv_33_results['param_estimator__epsilon']==epsilon]\
                            .sort_values('param_estimator__C').copy()
    
    display(cv_33_results_epsilon.head())
    
    # Name the variables to plot 
    cs=np.log10(cv_33_results_epsilon['param_estimator__C'])

    rank_te_R2_33=cv_33_results_epsilon['rank_test_R2']
    mean_te_R2_33=cv_33_results_epsilon['mean_test_R2']
    mean_tr_R2_33=cv_33_results_epsilon['mean_train_R2']
    std_te_R2_33=cv_33_results_epsilon['std_test_R2']
    std_tr_R2_33=cv_33_results_epsilon['std_train_R2']

    rank_te_MAE_33=cv_33_results_epsilon['rank_test_MAE']
    mean_te_MAE_33=cv_33_results_epsilon['mean_test_MAE']
    mean_tr_MAE_33=cv_33_results_epsilon['mean_train_MAE']
    std_te_MAE_33=cv_33_results_epsilon['std_test_MAE']
    std_tr_MAE_33=cv_33_results_epsilon['std_train_MAE']

    # Identify the optimal value
    idxmin_33=rank_te_R2_33.idxmin()
    c_idxmin_33=cs[idxmin_33]
    rank_te_R2_idxmin_33=rank_te_R2_33[idxmin_33]
    mean_te_R2_idxmin_33=mean_te_R2_33[idxmin_33]
    std_te_R2_idxmin_33=std_te_R2_33[idxmin_33]
    rank_te_MAE_idxmin_33=rank_te_MAE_33[idxmin_33]
    mean_te_MAE_idxmin_33=mean_te_MAE_33[idxmin_33]
    std_te_MAE_idxmin_33=std_te_MAE_33[idxmin_33]

    # Store rank 1 value for end comparison
    if rank_te_R2_idxmin_33 == 1:
        mean_te_MAE_idxmin_33_rank1=mean_te_MAE_idxmin_33
        std_te_MAE_idxmin_33_rank1=std_te_MAE_idxmin_33
        
        # Store the MAE score for each CV run for final comparison plot
        S33=cv_33_results.loc[idxmin_33,[c for c in cv_33_results.columns if ('_test_MAE' in c)&('split' in c)]]
        print(S33)
    
    # Plot mean scores
    plt.figure(figsize=(8,5))
    plt.plot(cs,mean_te_MAE_33, label='validation')
    plt.plot(cs,mean_tr_MAE_33, label='train')

    # Quantify variance of those scores with ±std curves
    plt.fill_between(cs,
                     mean_te_MAE_33-std_te_MAE_33,
                     mean_te_MAE_33+std_te_MAE_33,
                     alpha=0.2)
    plt.fill_between(cs,
                     mean_tr_MAE_33-std_tr_MAE_33,
                     mean_tr_MAE_33+std_tr_MAE_33,
                     alpha=0.2)

    # Add marker for best score
    plt.scatter(c_idxmin_33, mean_te_MAE_idxmin_33, marker='o', c='green', zorder=5)

    # Print best scores
    plt.title('Hyperparameter epsilon: {}\nHyperparameter C best: {:.4f}\nR2 score: {:.3f} \u00b1 {:.3f} (rank {:.0f})'\
              .format(epsilon,10**c_idxmin_33,mean_te_R2_idxmin_33,std_te_R2_idxmin_33,rank_te_R2_idxmin_33),
              loc='left')
    plt.annotate('MAE: {:.2f} \u00b1 {:.2f}'.format(mean_te_MAE_idxmin_33,std_te_MAE_idxmin_33),
                 xy=(c_idxmin_33, mean_te_MAE_idxmin_33),
                 xytext=(5*c_idxmin_33, 1.8*mean_te_MAE_idxmin_33),
                 arrowprops=dict(facecolor='green', shrink=0.05))
    plt.xlabel('$log_{10}(C)$')
    plt.ylabel('MAE')
    plt.legend(loc=2)
    plt.grid(color='black', linestyle='--', linewidth=0.1)
    plt.show()

We now plot the validation curve in 2D on a heatmap

In [ ]:
# Prepare the dataframe to display the 2D view of GridSearch Validation
pivot_df=cv_33_results[['param_estimator__C','param_estimator__epsilon','mean_test_MAE']].astype(float).copy()
#pd.options.display.float_format = '${:,.2f}'.format
pivot_df

In [ ]:
pivot_map=pivot_df.pivot(index='param_estimator__C', columns='param_estimator__epsilon', values='mean_test_MAE')
pivot_map

In [ ]:
# Display the heatmap of MAE scores on test test along the 2 hyperparameters dimensions

import matplotlib.ticker as mtick

fig, ax= plt.subplots(figsize=(8, 6))
heatmap=sns.heatmap(pivot_map,
                    vmin=pivot_map.min().min(), vmax=pivot_map.max().max(),
                    annot=False,
                    cmap='Blues')#,
                    #cbar_kws=dict(orientation='vertical')
heatmap.set_title('heatmap of MAE scores on validation set along the 2 hyperparameters dimensions',
                  fontdict={'fontsize':12}, pad=12)
plt.tick_params(axis='x', which='both',
                labelbottom = True,top=False, labeltop=False, labelrotation=90)

ax.xaxis.set_major_formatter(mtick.FormatStrFormatter('%.2e'))
ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2e'))
plt.show()

#### Models comparison

In [ ]:
print('Scores on the TRAIN & VALID set: \n')
print('MAE  model 0     Baseline mean {:>26.1f} EUR'.format(MAE_tr_model_0))
print('MAE  model 1     Linear Regression 3 features {:>11.1f} EUR'.format(MAE_tr_model_1))
print('MAE  model 2     Ridge {:>29.1f} \u00b1{:.1f} EUR'.format(mean_te_MAE_idxmin_2,std_te_MAE_idxmin_2))
print('MAE  model 3.1   kNN {:>31.1f} \u00b1{:.1f} EUR'.format(mean_te_MAE_idxmin_31,std_te_MAE_idxmin_31))
print('MAE  model 3.2   RandomForest {:>27.1f} EUR'.format(MAE_tr_model_32,1))
print('MAE  model 3.3   SVR {:>31.1f} \u00b1{:.1f} EUR'.format(mean_te_MAE_idxmin_33_rank1,std_te_MAE_idxmin_33_rank1))

print('\nScores on the TEST set: \n')
print('MAE  model 0     Baseline mean {:>26.1f} EUR'.format(MAE_te_model_0))
print('MAE  model 1     Linear Regression 3 features {:>11.1f} EUR'.format(MAE_te_model_1))
print('MAE  model 2     Ridge {:>34.1f} EUR'.format(MAE_te_model_2))
print('MAE  model 3.1   kNN {:>36.1f} EUR'.format(MAE_te_model_31))
print('MAE  model 3.2   RandomForest {:>27.1f} EUR'.format(MAE_te_model_32))
print('MAE  model 3.3   SVR {:>36.1f} EUR'.format(MAE_te_model_33))

print('\nScores on the OTHER TEST set: \n')
print('MAE  model 2     Ridge {:>34.1f} EUR'.format(MAE_o_model_2))

In [ ]:
plt.figure(figsize=(16,8))

# Gathering and plotting results on the train&validation set

S0=pd.Series(np.full_like(S2,MAE_tr_model_0))
S1=pd.Series(np.full_like(S2,MAE_tr_model_1))
S32=pd.Series(np.full_like(S2,MAE_tr_model_32))

models_df_tr=pd.concat([S0,S1,S2,S31,S32,S33],
                       axis=0,
                       keys=['Baseline (mean)','model 1 (Lr)',
                             'model 2 (Ridge)','model 3.1 (kNN)',
                             'model 3.2 (RF)','model 3.3 (SVR)'])

models_df_tr=models_df_tr.reset_index()

models_df_tr.columns=['Model','Run','MAE']

sns.boxplot(x='Model',y='MAE',data=models_df_tr, linewidth=0.5, palette='Set2')

sns.swarmplot(x='Model',y='MAE',data=models_df_tr)

# Gathering results and plotting on the test set

S0_te=pd.Series(MAE_te_model_0)
S1_te=pd.Series(MAE_te_model_1)
S2_te=pd.Series(MAE_te_model_2)
S31_te=pd.Series(MAE_te_model_31)
S32_te=pd.Series(MAE_te_model_32)
S33_te=pd.Series(MAE_te_model_33)

models_df_te=pd.concat([S0_te,S1_te,S2_te,S31_te,S32_te,S33_te],
                       axis=0,
                       keys=['Baseline (mean)','model 1 (Lr)',
                             'model 2 (Ridge)','model 3.1 (kNN)',
                             'model 3.2 (RF)','model 3.3 (SVR)'])

models_df_te=models_df_te.reset_index()

models_df_te.columns=['Model','Run','MAE']

sns.swarmplot(x='Model',y='MAE',data=models_df_te, color='red',
              marker='*',size=20, edgecolor='black', linewidth=1)

plt.grid()
plt.ylabel('MAE in EUR')
plt.show()

This chart gives an overview of the model performance, and it seems that the RF and the Ridge perform better.  
This is what we see when we play with the level of outliers detection.  
With the dataset we have now we manage to get test set prediction with Ridge at about MAE=10 at a minimum, and Ridge and RF are always close on the test set.

We now want to understand how the model predicts low or higher values of the target, and to see where the model perform: We want to understand which categories are better predicted. This is what we do in the next chapter.  

#### Check predictions

We feedback the data in the original dataframe and compare the prediction to the original true data.  

Again we generate a collection of plotting functions designed for this purpose.

In [ ]:
# Adding a column with index for merging purpose

df_clean_scrome_train['id']=df_clean_scrome_train.index
df_clean_scrome_test['id']=df_clean_scrome_test.index
df_clean_other_test['id']=df_clean_other_test.index

In [ ]:
# Merge predictions with original dataframes

df_clean_scrome_train=df_clean_scrome_train.merge(prediction_model_1_train_df,
                                how='left',on='id',suffixes=('', '_pred1'))\
                            .set_index('id',verify_integrity=True)

In [ ]:
df_clean_scrome_train=df_clean_scrome_train.merge(prediction_model_2_train_df,
                                how='left',on='id',suffixes=('', '_pred2'))\
                            .set_index('id',verify_integrity=True)

df_clean_scrome_train=df_clean_scrome_train.merge(prediction_model_31_train_df,
                                how='left',on='id',suffixes=('', '_pred31'))\
                            .set_index('id',verify_integrity=True)

df_clean_scrome_train=df_clean_scrome_train.merge(prediction_model_32_train_df,
                                how='left',on='id',suffixes=('', '_pred32'))\
                            .set_index('id',verify_integrity=True)

df_clean_scrome_train=df_clean_scrome_train.merge(prediction_model_33_train_df,
                                how='left',on='id',suffixes=('', '_pred33'))\
                            .set_index('id',verify_integrity=True)

In [ ]:
df_clean_scrome_test=df_clean_scrome_test.merge(prediction_model_1_test_df,
                                how='left',on='id',suffixes=('', '_pred1'))\
                            .set_index('id',verify_integrity=True)

In [ ]:
df_clean_scrome_test=df_clean_scrome_test.merge(prediction_model_2_test_df,
                                how='left',on='id',suffixes=('', '_pred2'))\
                            .set_index('id',verify_integrity=True)

df_clean_scrome_test=df_clean_scrome_test.merge(prediction_model_31_test_df,
                                how='left',on='id',suffixes=('', '_pred31'))\
                            .set_index('id',verify_integrity=True)

df_clean_scrome_test=df_clean_scrome_test.merge(prediction_model_32_test_df,
                                how='left',on='id',suffixes=('', '_pred32'))\
                            .set_index('id',verify_integrity=True)

df_clean_scrome_test=df_clean_scrome_test.merge(prediction_model_33_test_df,
                                how='left',on='id',suffixes=('', '_pred33'))\
                            .set_index('id',verify_integrity=True)

In [ ]:
df_clean_other_test=df_clean_other_test.merge(prediction_model_2_other_df,
                                how='left',on='id',suffixes=('', '_pred2'))\
                            .set_index('id',verify_integrity=True)

In [ ]:
df_clean_scrome_train.info(verbose=False)

In [ ]:
df_clean_scrome_test.info(verbose=False)

In [ ]:
df_clean_other_test.info(verbose=False)

We first plot the absolute values in Eur of the predictions in relation to the true value

In [ ]:
def check_scat_only_assy(cols):
    
    for c in cols:
        fig, ax = plt.subplots(2,3, figsize=(16,10), sharey=True)
        
        sns.scatterplot(x=10**df_clean_scrome_test[c],
                        y=df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred1'],
                        ax=ax[0,0], alpha=0.4)        
        ax[0,0].set_title('MODEL 1 - lr - Test data')

        sns.scatterplot(x=10**df_clean_scrome_test[c],
                        y=df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred2'],
                        ax=ax[0,1], alpha=0.4)
        ax[0,1].set_title('MODEL 2 - Ridge - Test data')  
        
        sns.scatterplot(x=10**df_clean_scrome_test[c],
                        y=df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred31'],
                        ax=ax[1,0], alpha=0.4)
        ax[1,0].set_title('MODEL 3.1 - kNN - Test data')    

        sns.scatterplot(x=10**df_clean_scrome_test[c],
                        y=df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred32'],
                        ax=ax[1,1], alpha=0.4)
        ax[1,1].set_title('MODEL 3.2 - RF - Test data')  

        sns.scatterplot(x=10**df_clean_scrome_test[c],
                        y=df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred33'],
                        ax=ax[1,2], alpha=0.4)
        ax[1,2].set_title('MODEL 3.3 - SVR - Test data')  

    plt.tight_layout()
    plt.show()

In [ ]:
if plotting_trigger==1:
    
    check_scat_only_assy(['cout_unitaire_pour_la_qte_demandee'])

We are then interested to plot the relative error of the predictions in relation to the true value.  
Those relative percentage errors are still massive but the system is in place are ready to include any improvement.  

In [ ]:
def check_scat_only_assy_ratio(cols):
    
    for c in cols:
        fig, ax = plt.subplots(2,3, figsize=(16,10), sharey=True)
        
        ratio_pred1=(10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']\
                         -df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred1'])\
                            /10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']
        sns.scatterplot(x=10**df_clean_scrome_test[c], y=ratio_pred1, ax=ax[0,0], alpha=0.4)
        ax[0,0].set_ylabel('( True - Prediction ) / True')
        ax[0,0].set_title('MODEL 1 - lr - Test data')      

        ratio_pred2=(10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']\
                         -df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred2'])\
                            /10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']              
        sns.scatterplot(x=10**df_clean_scrome_test[c],
                        y=ratio_pred2, ax=ax[0,1], alpha=0.4)     
        ax[0,1].set_ylabel('( True - Prediction ) / True')
        #ax[0,1].set_xlim(left=, right=)
        ax[0,1].set_ylim(bottom=-1, top=1)
        ax[0,1].set_title('MODEL 2 - Ridge - Test data')  
        
        ratio_pred31=(10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']\
                          -df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred31'])\
                            /10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']
        sns.scatterplot(x=10**df_clean_scrome_test[c],
                        y=ratio_pred31, ax=ax[1,0], alpha=0.4)       
        ax[1,0].set_ylabel('( True - Prediction ) / True')
        ax[1,0].set_title('MODEL 3.1 - kNN - Test data')   

        ratio_pred32=(10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']\
                          -df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred32'])\
                            /10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']
        sns.scatterplot(x=10**df_clean_scrome_test[c],
                        y=ratio_pred32, ax=ax[1,1], alpha=0.4)       
        ax[1,1].set_ylabel('( True - Prediction ) / True')
        ax[1,1].set_title('MODEL 3.2 - RF - Test data')

        ratio_pred33=(10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']\
                          -df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred33'])\
                            /10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']
        sns.scatterplot(x=10**df_clean_scrome_test[c],
                        y=ratio_pred33, ax=ax[1,2], alpha=0.4)     
        ax[1,2].set_ylabel('( True - Prediction ) / True')
        ax[1,2].set_title('MODEL 3.3 - SVR - Test data')    

    plt.tight_layout()
    plt.show()

In [ ]:
if plotting_trigger==1:
    
    check_scat_only_assy_ratio(['cout_unitaire_pour_la_qte_demandee'])

We perform the same plotting on our alternative test dataset.

In [ ]:
def check_scat_only_assy_o(cols):
    
    for c in cols:        
        sns.scatterplot(x=10**df_clean_other_test[c],
                        y=df_clean_other_test['cout_unitaire_pour_la_qte_demandee_pred2'],
                        alpha=0.4)      
        plt.title('MODEL 2 - Ridge - OTHER Test data')
        
    plt.tight_layout()
    plt.show()

In [ ]:
def check_scat_only_assy_ratio_o(cols):
    
    for c in cols:
        ratio_pred2=(10**df_clean_other_test['cout_unitaire_pour_la_qte_demandee']\
                         -df_clean_other_test['cout_unitaire_pour_la_qte_demandee_pred2'])\
                            /10**df_clean_other_test['cout_unitaire_pour_la_qte_demandee']              
        sns.scatterplot(x=10**df_clean_other_test[c], y=ratio_pred2, alpha=0.4)     
        plt.ylabel('( True - Prediction ) / True')
        plt.title('MODEL 2 - Ridge - OTHER Test data')  
        
    plt.tight_layout()
    plt.show()

In [ ]:
if plotting_trigger==1:
        check_scat_only_assy_o(['cout_unitaire_pour_la_qte_demandee'])
        check_scat_only_assy_ratio_o(['cout_unitaire_pour_la_qte_demandee'])

We see that our relative erros are generally still high.

#### Analysis of the errors

We want to understand the origin of errors better.
To do this we will analyse the points with most and least important mean absolute errors and see their position on the various features distribution.
We do this on the test set, it would also be interesting to analyse the train set in this respect.  

In [ ]:
# We add columns corresponding to the MAE calculated on each datapoint
# rapported to the data value

df_clean_scrome_test['error_pred1']\
            =np.abs(10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']\
            -df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred1'])\
            /10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']

df_clean_scrome_test['error_pred2']\
            =np.abs(10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']\
            -df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred2'])\
            /10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']

df_clean_scrome_test['error_pred31']\
            =np.abs(10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']\
            -df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred31'])\
            /10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']

df_clean_scrome_test['error_pred32']\
            =np.abs(10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']\
            -df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred32'])\
            /10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']

df_clean_scrome_test['error_pred33']\
            =np.abs(10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']\
            -df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred33'])\
            /10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']

df_clean_other_test['error_pred1']\
            =np.abs(10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']\
            -df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred1'])\
            /10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']

df_clean_other_test['error_pred2']\
            =np.abs(10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']\
            -df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred2'])\
            /10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']

df_clean_other_test['error_pred31']\
            =np.abs(10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']\
            -df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred31'])\
            /10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']

df_clean_other_test['error_pred32']\
            =np.abs(10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']\
            -df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred32'])\
            /10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']

df_clean_other_test['error_pred33']\
            =np.abs(10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']\
            -df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred33'])\
            /10**df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee']

In [ ]:
# We identify the top and lowest errors, which we want to analyse further

df_clean_scrome_test_head=\
                    df_clean_scrome_test.sort_values('error_pred2',
                                                     ascending=False).head(20)

df_clean_scrome_test_tail=\
                    df_clean_scrome_test.sort_values('error_pred2',
                                                     ascending=False).tail(20)

In [ ]:
# Display project with top errors

df_clean_scrome_test_head

In [ ]:
# Display project with lowest errors

df_clean_scrome_test_tail

In [ ]:
# Visualize the projects corresponding to head

print('HEAD')
display(df_merged[df_merged.index.isin(df_clean_scrome_test_head.index)]['calc_file'])

# Display project drawing and informations
for project_id in list(df_clean_scrome_test_head.index):
    print('Project {}:'.format(project_id))
    project_id_pdf_path=os.path.normpath(df_merged[df_merged.index==project_id]['pdf_file'].item())
    if project_id_pdf_path[-2]=='d': 
        project_id_img_path=project_id_pdf_path[:-4]+'_page1from*_pre.jpg'
    else:
        project_id_img_path=project_id_pdf_path[:-5]+'_page1from*_pre.jpg'
    #print('{}'.format(project_id_img_path))
    jn=glob.glob(os.path.normpath(project_id_img_path))[0]
    print(jn)
    im=Image.open(jn)
    size=800,800
    im.thumbnail(size, Image.ANTIALIAS)
    display(im)
    display(pd.DataFrame(X_te[list(X_te_index).index(project_id),:], index=df_clean_scrome_cols).T)

In [ ]:
# Visualize the projects corresponding to tail

print('TAIL')
display(df_merged[df_merged.index.isin(df_clean_scrome_test_tail.index)]['calc_file'])

# Display project drawing and informations
for project_id in list(df_clean_scrome_test_tail.index):
    print('Project {}:'.format(project_id))
    project_id_pdf_path=os.path.normpath(df_merged[df_merged.index==project_id]['pdf_file'].item())
    if project_id_pdf_path[-2]=='d': 
        project_id_img_path=project_id_pdf_path[:-4]+'_page1from*.jpg'
    else:
        project_id_img_path=project_id_pdf_path[:-5]+'_page1from*.jpg'
    #print('{}'.format(project_id_img_path))
    jn=glob.glob(os.path.normpath(project_id_img_path))[0]
    print(jn)
    im=Image.open(jn)
    size=800,800
    im.thumbnail(size, Image.ANTIALIAS)
    display(im)
    display(pd.DataFrame(X_te[list(X_te_index).index(project_id),:], index=df_clean_scrome_cols).T)

In [ ]:
# Preparing dataframe for EDA on errors

df_clean_scrome_test['error_pred2_headtail']='core'

df_clean_scrome_test.loc[df_clean_scrome_test_tail.index,
                         'error_pred2_headtail']='tail'

df_clean_scrome_test.loc[df_clean_scrome_test_head.index,
                         'error_pred2_headtail']='head'

df_clean_scrome_test_topfeat=\
        df_clean_scrome_test[feature_names+['error_pred2_headtail']]\
        .astype(float, errors='ignore').copy()

df_clean_scrome_test_allfeat=\
        df_clean_scrome_test\
        .astype(float, errors='ignore').copy()

In [ ]:
# Plotting position of head and tail of errors group on feature distribution

if plotting_trigger == 1:
    
    for c in df_clean_scrome_test_topfeat.columns:#df_clean_scrome_cols
        
        x=df_clean_scrome_test_allfeat[c]
        
        sns.displot(x=x,kind='hist',
                    hue=df_clean_scrome_test_allfeat['error_pred2_headtail'],
                    multiple='stack',
                    bins=20,
                    hue_order=['core','head','tail'],
                    palette=['white','r','g'])
        
        plt.title('Position errors head & tail on feature distribution')
        
        plt.show()

From those plots we do not see clear situations where the models fail or succeed in predicting the target.  
The head and tail elements seem to be spread in the dataset.

#### Additional plots

Let's prepare for plotting of additional insights: the location of the predicted points on the test set per feature in comparison to the train set.

In [ ]:
def check_scat_assy(cols):
    
    for c in cols:
        if df_clean_scrome_train[c].dtype==np.number:
            fig, ax = plt.subplots(3,4, figsize=(16,12))

            sns.distplot(df_clean_scrome_train[c],
                         ax=ax[0,0],
                         bins=50,
                         fit=None,
                         color='grey',
                         norm_hist=True,
                         hist=False,
                         label='Train data - Original')
            sns.distplot(df_clean_scrome_test[c],
                         ax=ax[0,0],
                         bins=50,
                         fit=None,
                         color='g',
                         norm_hist=True,
                         hist=False,
                         label='Test data - Original')
            ax[0,0].set_title('{}'.format(c))
            ax[0,0].legend()

            sns.kdeplot(x=df_clean_scrome_train[c],
                        y=df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee_pred33'],
                        color='m',
                        ax=ax[0,1],
                        label='Train data - Prediction model 33',
                        fill=False,
                        alpha=0.4)               
            sns.kdeplot(x=df_clean_scrome_train[c],
                        y=df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee_pred32'],
                        color='y',
                        ax=ax[0,1],
                        label='Train data - Prediction model 32',
                        fill=False,
                        alpha=0.4)               
            sns.kdeplot(x=df_clean_scrome_train[c],
                        y=df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee_pred31'],
                        color='b',
                        ax=ax[0,1],
                        label='Train data - Prediction model 31',
                        fill=False,
                        alpha=0.4)               
            sns.kdeplot(x=df_clean_scrome_train[c],
                        y=df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee_pred2'],
                        color='r',
                        ax=ax[0,1],
                        label='Train data - Prediction model 2',
                        fill=False,
                        alpha=0.4)
            sns.kdeplot(x=df_clean_scrome_train[c],
                        y=df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee_pred1'],
                        color='g',
                        ax=ax[0,1],
                        label='Train data - Prediction model 1',
                        fill=False,
                        alpha=0.4)
            ax[0,1].set_title('TRAIN {}'.format(c))  
            ax[0,1].legend()

            sns.kdeplot(x=df_clean_scrome_test[c],
                        y=df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred33'],
                        color='m',
                        ax=ax[0,2],
                        label='Test data - Prediction model 33',
                        fill=False,
                        alpha=0.4)
            sns.kdeplot(x=df_clean_scrome_test[c],
                        y=df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred32'],
                        color='y',
                        ax=ax[0,2],
                        label='Test data - Prediction model 32',
                        fill=False,
                        alpha=0.4)
            sns.kdeplot(x=df_clean_scrome_test[c],
                        y=df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred31'],
                        color='b',
                        ax=ax[0,2],
                        label='Test data - Prediction model 31',
                        fill=False,
                        alpha=0.4)               
            sns.kdeplot(x=df_clean_scrome_test[c],
                        y=df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred2'],
                        color='r',
                        ax=ax[0,2],
                        label='Test data - Prediction model 2',
                        fill=False,
                        alpha=0.4)
            sns.kdeplot(x=df_clean_scrome_test[c],
                        y=df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred1'],
                        color='g',
                        ax=ax[0,2],
                        label='Test data - Prediction model 1',
                        fill=False,
                        alpha=0.4)
            ax[0,2].set_title('TEST {}'.format(c))  
            ax[0,2].legend()        


            sns.kdeplot(x=df_clean_scrome_train[c],
                        y=df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee_pred2'],
                        color='grey', ax=ax[1,0], label='Train data - Prediction model 2',
                        shade=True,
                        alpha=0.4)               
            sns.kdeplot(x=df_clean_scrome_test[c],
                        y=df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred2'],
                        color='r',
                        ax=ax[1,0],
                        label='Test data - Prediction model 2',
                        shade=True,
                        alpha=0.4)        
            ax[1,0].set_title('{}'.format(c))  
            ax[1,0].legend()

            sns.kdeplot(x=df_clean_scrome_train[c],
                        y=df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee_pred31'],
                        color='grey',
                        ax=ax[1,1],
                        label='Train data - Prediction model 31',
                        shade=True,
                        alpha=0.4)               
            sns.kdeplot(x=df_clean_scrome_test[c],
                        y=df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred31'],
                        color='b',
                        ax=ax[1,1],
                        label='Test data - Prediction model 31',
                        shade=True,
                        alpha=0.4)        
            ax[1,1].set_title('{}'.format(c))  
            ax[1,1].legend()

            sns.kdeplot(x=df_clean_scrome_train[c],
                        y=df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee_pred32'],
                        color='grey',
                        ax=ax[1,2],
                        label='Train data - Prediction model 32',
                        shade=True,
                        alpha=0.4)               
            sns.kdeplot(x=df_clean_scrome_test[c],
                        y=df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred32'],
                        color='y',
                        ax=ax[1,2],
                        label='Test data - Prediction model 32',
                        shade=True,
                        alpha=0.4)        
            ax[1,2].set_title('{}'.format(c))  
            ax[1,2].legend()
            
            sns.kdeplot(x=df_clean_scrome_train[c],
                        y=df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee_pred33'],
                        color='grey',
                        ax=ax[1,3],
                        label='Train data - Prediction model 33',
                        shade=True,
                        alpha=0.4)               
            sns.kdeplot(x=df_clean_scrome_test[c],
                        y=df_clean_scrome_test['cout_unitaire_pour_la_qte_demandee_pred33'],
                        color='m',
                        ax=ax[1,3],
                        label='Test data - Prediction model 33',
                        shade=True,
                        alpha=0.4)        
            ax[1,3].set_title('{}'.format(c))  
            ax[1,3].legend()

            sns.scatterplot(x=df_clean_scrome_train[c],
                            y=10**df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee'],
                            color='grey',
                            ax=ax[2,0],
                            label='Train data - Original',
                            alpha=0.2)#,shade=True,                
            sns.scatterplot(x=df_clean_scrome_train[c],
                            y=df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee_pred2'],
                            color='r',
                            ax=ax[2,0],
                            label='Train data - Prediction model 2',
                            alpha=0.2)#,shade=True        
            ax[2,0].set_title('{}'.format(c))  
            ax[2,0].legend()

            sns.scatterplot(x=df_clean_scrome_train[c],
                            y=10**df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee'],
                            color='grey',
                            ax=ax[2,1],
                            label='Train data - Original',
                            alpha=0.2)#,shade=True,                
            sns.scatterplot(x=df_clean_scrome_train[c],
                            y=df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee_pred31'],
                            color='b', 
                            ax=ax[2,1],
                            label='Train data - Prediction model 31',
                            alpha=0.2)#,shade=True        
            ax[2,1].set_title('{}'.format(c))  
            ax[2,1].legend()      

            sns.scatterplot(x=df_clean_scrome_train[c],
                            y=10**df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee'],
                            color='grey',
                            ax=ax[2,2],
                            label='Train data - Original',
                            alpha=0.2)#,shade=True,                
            sns.scatterplot(x=df_clean_scrome_train[c],
                            y=df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee_pred32'],
                            color='y',
                            ax=ax[2,2],
                            label='Train data - Prediction model 32',
                            alpha=0.2)#,shade=True        
            ax[2,2].set_title('{}'.format(c))  
            ax[2,2].legend()

            sns.scatterplot(x=df_clean_scrome_train[c],
                            y=10**df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee'],
                            color='grey',
                            ax=ax[2,3],
                            label='Train data - Original',
                            alpha=0.2)#,shade=True,                
            sns.scatterplot(x=df_clean_scrome_train[c],
                            y=df_clean_scrome_train['cout_unitaire_pour_la_qte_demandee_pred33'],
                            color='m',
                            ax=ax[2,3],
                            label='Train data - Prediction model 33',
                            alpha=0.2)#,shade=True        
            ax[2,3].set_title('{}'.format(c))  
            ax[2,3].legend()

            
        plt.tight_layout()
        plt.show()

        
if plotting_trigger==2:

    print(quant_features)    
    #bool_features
    #cat_features
    
    for c in quant_features:
        check_scat_assy([c])

### (d) Predictions - Conclusion

With this setup we can reach and average MAE of around 20Eur, depending on the conditions, and we obtain predictions on the test sets in a ±50% range when we would need a value in the range of ±5%, an acceptable error in relation to a typical project margin.

We now have a system in place to perform analysis on our datapoint and try to increase the score of our models. We have the following ideas for continuation:  

We see a possibility to increase the quality of our prediction on the target discussed by incorporating more information in our features, mainly more symbols and the step source. For the more symbols we need a detector, for the steps files, we need to wait the our parnter generates the data.  

We also see as an additional topic of investigation the different levels of score for the different targets, such as internal costs, or machining cost alone, instead of taking as target the total project costs. Perhaps we could see that the model does not predict well some portion of the cost - say the external ones - but can predict correctly some other portion of the costs - say the internal ones.    